In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
data_in = np.load("a2_data/images.npy")
data_out = np.load("a2_data/labels.npy")

In [3]:
X_valid, X_train = data_in[15000:]/np.amax(data_in), data_in[:15000]/np.amax(data_in)
y_valid, y_train = data_out[15000:], data_out[:15000]

In [4]:
input_ = keras.layers.Input(shape=(150,150,1))
cnn = keras.layers.Conv2D(50, kernel_size=7, strides=2, activation='relu')(input_)
cnn = keras.layers.MaxPooling2D(pool_size=(2,2), strides=2)(cnn)
cnn = keras.layers.Conv2D(100, kernel_size=3, strides=1, activation='relu')(cnn)
cnn = keras.layers.MaxPooling2D(pool_size=(2,2))(cnn)
cnn = keras.layers.Conv2D(200, kernel_size=3, strides=1, activation='relu')(cnn)
cnn = keras.layers.MaxPooling2D(pool_size=(2,2))(cnn)
cnn = keras.layers.Conv2D(400, kernel_size=3, strides=1, activation='relu')(cnn)
cnn = keras.layers.Dropout(0.6)(cnn)
cnn = keras.layers.Flatten()(cnn)

hour = keras.layers.Dense(144, activation='relu')(cnn)
hour = keras.layers.Dense(144, activation='relu')(hour)
hour = keras.layers.Dense(12, activation='softmax', name='hour')(hour)

minute = keras.layers.Dense(100, activation='relu')(cnn)
minute = keras.layers.Dense(100, activation='relu')(minute)
minute = keras.layers.Dense(1, activation='linear', name='minute')(minute)

model = keras.Model(inputs=input_, outputs=[hour, minute])

Metal device set to: Apple M1


2022-11-10 14:02:43.563824: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-10 14:02:43.568776: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
adam = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss=['sparse_categorical_crossentropy', 'mse'], 
              optimizer=adam, metrics=['accuracy', 'mae'])

In [ ]:
history = model.fit(X_train, [y_train[:,0], y_train[:,1]], epochs=20, 
                    validation_data=(X_valid, [y_valid[:,0], y_valid[:,1]]))

2022-11-10 14:02:49.727592: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/20


2022-11-10 14:02:51.105135: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


469/469 [==============================] - ETA: 0s - loss: 339.0472 - hour_loss: 2.3414 - minute_loss: 336.7057 - hour_accuracy: 0.1009 - hour_mae: 4.4333 - minute_accuracy: 0.0167 - minute_mae: 15.6135

2022-11-10 14:05:09.852128: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


469/469 [==============================] - 149s 314ms/step - loss: 339.0472 - hour_loss: 2.3414 - minute_loss: 336.7057 - hour_accuracy: 0.1009 - hour_mae: 4.4333 - minute_accuracy: 0.0167 - minute_mae: 15.6135 - val_loss: 312.8989 - val_hour_loss: 6.8108 - val_minute_loss: 306.0880 - val_hour_accuracy: 0.0000e+00 - val_hour_mae: 10.4167 - val_minute_accuracy: 0.0167 - val_minute_mae: 15.0963
Epoch 2/20
469/469 [==============================] - 146s 312ms/step - loss: 310.9762 - hour_loss: 2.3086 - minute_loss: 308.6678 - hour_accuracy: 0.0985 - hour_mae: 4.4333 - minute_accuracy: 0.0167 - minute_mae: 15.1194 - val_loss: 311.6856 - val_hour_loss: 8.7390 - val_minute_loss: 302.9465 - val_hour_accuracy: 0.0000e+00 - val_hour_mae: 10.4167 - val_minute_accuracy: 0.0167 - val_minute_mae: 15.0356
Epoch 3/20
469/469 [==============================] - 144s 307ms/step - loss: 307.8346 - hour_loss: 2.3068 - minute_loss: 305.5277 - hour_accuracy: 0.0963 - hour_mae: 4.4333 - minute_accuracy: 0.01

Error: command buffer exited with error status.
	The Metal Performance Shaders operations encoded on it may not have completed.
	Error: 
	(null)
	Internal Error (0000000e:Internal Error)
	<AGXG13GFamilyCommandBuffer: 0x397d2c060>
    label = <none> 
    device = <AGXG13GDevice: 0x1453a0e00>
        name = Apple M1 
    commandQueue = <AGXG13GFamilyCommandQueue: 0x121419600>
        label = <none> 
        device = <AGXG13GDevice: 0x1453a0e00>
            name = Apple M1 
    retainedReferences = 1


469/469 [==============================] - 182s 388ms/step - loss: 305.5704 - hour_loss: 2.3045 - minute_loss: 303.2657 - hour_accuracy: 0.1003 - hour_mae: 4.4333 - minute_accuracy: 0.0167 - minute_mae: 15.0380 - val_loss: 322.6082 - val_hour_loss: 9.9862 - val_minute_loss: 312.6219 - val_hour_accuracy: 0.0000e+00 - val_hour_mae: 10.4167 - val_minute_accuracy: 0.0167 - val_minute_mae: 15.2086
Epoch 8/20
469/469 [==============================] - 156s 332ms/step - loss: 304.7308 - hour_loss: 2.3046 - minute_loss: 302.4263 - hour_accuracy: 0.0955 - hour_mae: 4.4333 - minute_accuracy: 0.0167 - minute_mae: 15.0330 - val_loss: 310.8029 - val_hour_loss: 10.6303 - val_minute_loss: 300.1726 - val_hour_accuracy: 0.0000e+00 - val_hour_mae: 10.4167 - val_minute_accuracy: 0.0167 - val_minute_mae: 15.0002
Epoch 9/20
367/469 [======================>.......] - ETA: 29s - loss: 324.5864 - hour_loss: 2.3048 - minute_loss: 322.2819 - hour_accuracy: 0.0991 - hour_mae: 4.4241 - minute_accuracy: 0.0163 - m

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0,1000) # set the vertical range to [0-1]
plt.gca().set_xlim(0,29)
plt.show()